In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
tf.test.is_built_with_cuda()

True

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle
import cupy as cp

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

# Data Fetching

In [3]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16])
    out= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19])
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# Min Max Scaler

In [4]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(A1)
Y1=scaler_obj.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [6]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Parameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

def hyperparameterRF(x_train,y_train):
    random_grid = {'bootstrap': [True, False],
     'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
    }
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

    grid_result = MultiOutputRegressor(rf_random).fit(x_train, y_train)
    
    return grid_result.estimators_[0].best_params_
    

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.5,random_state=0)

params=hyperparameterRF(x_train,y_train)
params

# Model

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=0)

inp=keras.Input(shape=(1,5))

l=keras.layers.Conv1D(16,1,padding="same",activation="relu",kernel_initializer="glorot_uniform",bias_initializer="zeros",
                        kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=keras.regularizers.l2(1e-4),
                        activity_regularizer=keras.regularizers.l2(1e-5))(inp)

l=keras.layers.Conv1D(64,4,padding="same",activation="sigmoid")(l)
l=keras.layers.Dropout(.2)(l)
l=keras.layers.Dense(64, activation="relu")(l)

l=keras.layers.LSTM(256,activation="tanh",use_bias=True,kernel_initializer="glorot_uniform",bias_initializer="zeros",
                       return_sequences = True)(l)
l=keras.layers.LSTM(128, activation = "tanh", return_sequences = False)(l)
l=keras.layers.Dropout(0.2)(l)
output=keras.layers.Dense(7)(l)

model9=keras.Model(inputs=inp,outputs=output)
model9.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='mse',metrics=['accuracy',rmse,coeff_determination])
model9.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 5)]            0         
                                                                 
 conv1d (Conv1D)             (None, 1, 16)             96        
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 64)             4160      
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 dense (Dense)               (None, 1, 64)             4160      
                                                                 
 lstm (LSTM)                 (None, 1, 256)            328704    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120

In [9]:
model_fit8=model9.fit(x_train,y_train,batch_size=256,epochs=10,validation_data=(x_test, y_test),shuffle= False)

Epoch 1/10
5070/5070 [==============================] - 139s 27ms/step - loss: 0.0010 - accuracy: 0.9438 - rmse: 0.0086 - coeff_determination: -61.3173 - val_loss: 8.9378e-04 - val_accuracy: 0.9437 - val_rmse: 0.0075 - val_coeff_determination: -54.8860
Epoch 2/10
5070/5070 [==============================] - 125s 25ms/step - loss: 7.9844e-04 - accuracy: 0.9438 - rmse: 0.0078 - coeff_determination: -56.5195 - val_loss: 6.9955e-04 - val_accuracy: 0.9437 - val_rmse: 0.0075 - val_coeff_determination: -54.6812
Epoch 3/10
5070/5070 [==============================] - 140s 28ms/step - loss: 6.2074e-04 - accuracy: 0.9438 - rmse: 0.0075 - coeff_determination: -55.5562 - val_loss: 5.4205e-04 - val_accuracy: 0.9437 - val_rmse: 0.0075 - val_coeff_determination: -54.6991
Epoch 4/10
5070/5070 [==============================] - 129s 25ms/step - loss: 4.7928e-04 - accuracy: 0.9438 - rmse: 0.0075 - coeff_determination: -55.3531 - val_loss: 4.1648e-04 - val_accuracy: 0.9437 - val_rmse: 0.0075 - val_coeff_

# Saving Model as File

In [ ]:
model_json = model8.to_json()
with open("lstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model8.save_weights("lstm.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json
json_file = open('lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['accuracy','mse','mae',rmse,coeff_determination])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

# Error Analysis

In [ ]:
_, train_acc = model8.evaluate(x_train, y_train, verbose=0)
_, test_acc = model8.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# summarize history for loss
plt.plot(model_fit8.history['loss'])
plt.plot(model_fit8.history['val_loss'])
plt.title('Model Loss',fontweight ='bold',fontsize = 15)
plt.ylabel('Loss',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(model_fit8.history['accuracy'])
plt.plot(model_fit8.history['val_accuracy'])
plt.title('Model accuracy',fontweight ='bold',fontsize = 15)
plt.ylabel('Accuracy',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()